## 4: FIRST FILTER - FOR HOW MUCH WILL THEY SHOP?


1. CHECK THE CORRELATIONS WITH ORDER PRICE AND AMOUNT
2. WHAT DO THEY MEAN?
3. USE AS FILTER TO LIMIT AMOUNT OF GROCERIES PER DAY/STORE/TIME/...

### Import libraries 

In [2]:
#Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Matplotlib is a plotting library for python and pyplot gives us a MatLab like plotting framework. We will use this in our plotter function to plot data.
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import StrMethodFormatter
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns
import dataframe_image as dfi

from scipy.stats import f_oneway

### Load and view data 

In [3]:
df = pd.read_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2.csv")

# del df["HH"]
df.describe(include='all')

,week,order_ID,item_name,amount,price_unit,price_total,item_type,category,day,time,...,storetype_num,storename_num,cat_num,time_num,datetime,timestamp,dates,times,times_min,dates_days
count,372.000000,372.000000,372,372.000000,372.000000,372.000000,372,372,372,372,...,372.000000,372.000000,372.000000,372.000000,372,3.720000e+02,372,372,372.000000,3.720000e+02
unique,NaN,NaN,314,NaN,NaN,NaN,139,16,7,4,...,NaN,NaN,NaN,NaN,40,NaN,26,35,NaN,NaN
top,NaN,NaN,GROF BROOD GESN.,NaN,NaN,NaN,charcuterie,fruit & vegetables,Saturday,morning,...,NaN,NaN,NaN,NaN,2021-11-26 17:25:00,NaN,2022-01-08,17:25:00,NaN,NaN
freq,NaN,NaN,6,NaN,NaN,NaN,25,103,87,186,...,NaN,NaN,NaN,NaN,44,NaN,45,44,NaN,NaN
mean,4.043011,18.881720,NaN,1.274194,2.602328,2.867247,NaN,NaN,NaN,NaN,...,3.392473,4.212366,7.462366,1.387097,NaN,1.639888e+18,NaN,NaN,1353.930108,2.021425e+07
std,2.478648,10.908193,NaN,1.103819,1.947725,2.019200,NaN,NaN,NaN,NaN,...,1.344550,2.396214,4.079557,0.896621,NaN,1.924092e+15,NaN,NaN,371.187009,4.252837e+03
min,1.000000,1.000000,NaN,1.000000,0.064680,0.064680,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,NaN,1.637435e+18,NaN,NaN,802.000000,2.021112e+07
25%,1.000000,8.000000,NaN,1.000000,1.300000,1.543970,NaN,NaN,NaN,NaN,...,4.000000,3.000000,6.000000,1.000000,NaN,1.637948e+18,NaN,NaN,1055.000000,2.021113e+07
50%,4.000000,20.000000,NaN,1.000000,2.240250,2.462860,NaN,NaN,NaN,NaN,...,4.000000,4.000000,9.000000,2.000000,NaN,1.639306e+18,NaN,NaN,1179.500000,2.021121e+07
75%,6.000000,26.250000,NaN,1.000000,3.290000,3.875000,NaN,NaN,NaN,NaN,...,4.000000,6.000000,11.000000,2.000000,NaN,1.641640e+18,NaN,NaN,1725.000000,2.022011e+07


## Check the correlations

## Category count vs. store/day/time

In [4]:
df['count_cat'] = df.groupby('order_ID')['category'].transform('nunique')

In [5]:
df_orders = df[['order_ID', 'store_name', 'storename_num', 'store_type', 'storetype_num','day', 'day_num', 'time', 'time_num', 'timestamp', 'times', 'dates', 'times_min', 'dates_days', 'order_amount', 'order_price', 'count_cat']]
df_orders = df_orders.drop_duplicates()

In [8]:
# f_oneway() function takes the group data as input and returns F-statistic and P-value
 
# Assumption(H0) is that cat and storename  are NOT correlated
 
# Finds out the Prices data for each FuelType as a list
CategoryGroupLists=df_orders.groupby('store_type')['count_cat'].apply(list)
CategoryGroupLists

store_type
bakery                          [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
butcher                                                 [1, 1, 2, 1]
drugstore                                                        [3]
furniture store                                                  [1]
supermarket        [7, 3, 1, 7, 1, 12, 2, 8, 6, 1, 9, 7, 8, 2, 2,...
Name: count_cat, dtype: object

In [9]:
# Performing the ANOVA test
# We accept the Assumption(H0) only when P-Value &gt; 0.05
AnovaResults = f_oneway(*CategoryGroupLists)
print('P-Value for Anova is: ', AnovaResults[1])

P-Value for Anova is:  0.026940694757497417


In [6]:
# f_oneway() function takes the group data as input and returns F-statistic and P-value
 
# Assumption(H0) is that cat and storename  are NOT correlated
 
# Finds out the Prices data for each FuelType as a list
CategoryGroupLists=df_orders.groupby('store_name')['count_cat'].apply(list)
CategoryGroupLists

store_name
Albert Heijn        [1, 1, 2, 1, 2, 2, 2, 1, 9, 1, 2, 1]
Brabo                                                [1]
Carrefour                                            [7]
Delhaize                                       [7, 6, 8]
Ikea                                                 [1]
Kruidvat                                             [3]
Okay                                 [3, 12, 8, 9, 7, 3]
Sys                 [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]
Versavel Poelman                               [1, 2, 1]
Name: count_cat, dtype: object

In [7]:
# Performing the ANOVA test
# We accept the Assumption(H0) only when P-Value &gt; 0.05
AnovaResults = f_oneway(*CategoryGroupLists)
print('P-Value for Anova is: ', AnovaResults[1])

P-Value for Anova is:  3.0415219640292695e-05


# New dataframes for cat count per store name

In [19]:
# to select only one
store1 = 'Albert Heijn'
store2 = 'Sys'
store3 = 'Okay'
store4 = 'Delhaize'
store5 = 'Versavel Poelman'
store6 = 'Kruidvat'
store7 = 'Brabo'
store8 = 'Ikea'
store9 = 'Carrefour'

# select only these data for the df
df_AH = df_orders[df_orders['store_name'] == store1]
df_SYS = df_orders[df_orders['store_name'] == store2]
df_OKAY = df_orders[df_orders['store_name'] == store3]
df_DELHAIZE = df_orders[df_orders['store_name'] == store4]
df_VERSAVEL = df_orders[df_orders['store_name'] == store5]
df_KRUIDVAT = df_orders[df_orders['store_name'] == store6]
df_BRABO = df_orders[df_orders['store_name'] == store7]
df_IKEA = df_orders[df_orders['store_name'] == store8]
df_CARREFOUR = df_orders[df_orders['store_name'] == store9]

In [20]:
df_AH = df_AH.groupby(['count_cat'])['order_ID'].nunique()
df_AH = pd.DataFrame (df_AH)
df_AH = df_AH.reset_index()
df_AH = df_AH.loc[~(df_AH==0).any(axis=1)]
df_AH = df_AH.rename(columns={"order_ID": "weight"})

In [21]:
df_SYS = df_SYS.groupby(['count_cat'])['order_ID'].nunique()
df_SYS = pd.DataFrame (df_SYS)
df_SYS = df_SYS.reset_index()
df_SYS = df_SYS.loc[~(df_SYS==0).any(axis=1)]
df_SYS = df_SYS.rename(columns={"order_ID": "weight"})

In [22]:
df_OKAY = df_OKAY.groupby(['count_cat'])['order_ID'].nunique()
df_OKAY = pd.DataFrame (df_OKAY)
df_OKAY = df_OKAY.reset_index()
df_OKAY = df_OKAY.loc[~(df_OKAY==0).any(axis=1)]
df_OKAY = df_OKAY.rename(columns={"order_ID": "weight"})

In [23]:
df_DELHAIZE = df_DELHAIZE.groupby(['count_cat'])['order_ID'].nunique()
df_DELHAIZE = pd.DataFrame (df_DELHAIZE)
df_DELHAIZE = df_DELHAIZE.reset_index()
df_DELHAIZE = df_DELHAIZE.loc[~(df_DELHAIZE==0).any(axis=1)]
df_DELHAIZE = df_DELHAIZE.rename(columns={"order_ID": "weight"})

In [24]:
df_VERSAVEL = df_VERSAVEL.groupby(['count_cat'])['order_ID'].nunique()
df_VERSAVEL = pd.DataFrame (df_VERSAVEL)
df_VERSAVEL = df_VERSAVEL.reset_index()
df_VERSAVEL = df_VERSAVEL.loc[~(df_VERSAVEL==0).any(axis=1)]
df_VERSAVEL = df_VERSAVEL.rename(columns={"order_ID": "weight"})

In [25]:
df_KRUIDVAT = df_KRUIDVAT.groupby(['count_cat'])['order_ID'].nunique()
df_KRUIDVAT = pd.DataFrame (df_KRUIDVAT)
df_KRUIDVAT = df_KRUIDVAT.reset_index()
df_KRUIDVAT = df_KRUIDVAT.loc[~(df_KRUIDVAT==0).any(axis=1)]
df_KRUIDVAT = df_KRUIDVAT.rename(columns={"order_ID": "weight"})

In [26]:
df_BRABO = df_BRABO.groupby(['count_cat'])['order_ID'].nunique()
df_BRABO = pd.DataFrame (df_BRABO)
df_BRABO = df_BRABO.reset_index()
df_BRABO = df_BRABO.loc[~(df_BRABO==0).any(axis=1)]
df_BRABO = df_BRABO.rename(columns={"order_ID": "weight"})

In [27]:
df_IKEA = df_IKEA.groupby(['count_cat'])['order_ID'].nunique()
df_IKEA = pd.DataFrame (df_IKEA)
df_IKEA = df_IKEA.reset_index()
df_IKEA = df_IKEA.loc[~(df_IKEA==0).any(axis=1)]
df_IKEA = df_IKEA.rename(columns={"order_ID": "weight"})

In [28]:
df_CARREFOUR = df_CARREFOUR.groupby(['count_cat'])['order_ID'].nunique()
df_CARREFOUR = pd.DataFrame (df_CARREFOUR)
df_CARREFOUR = df_CARREFOUR.reset_index()
df_CARREFOUR = df_CARREFOUR.loc[~(df_CARREFOUR==0).any(axis=1)]
df_CARREFOUR = df_CARREFOUR.rename(columns={"order_ID": "weight"})

In [29]:
df_CARREFOUR

,count_cat,weight
0,7,1


In [30]:
df_AH.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2_AHcatcount.csv", index = None, header=True)
df_SYS.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2_SYcatcount.csv", index = None, header=True)
df_OKAY.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2_OKcatcount.csv", index = None, header=True)
df_DELHAIZE.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2_DEcatcount.csv", index = None, header=True)
df_VERSAVEL.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2_VEcatcount.csv", index = None, header=True)
df_KRUIDVAT.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2_KRcatcount.csv", index = None, header=True)
df_BRABO.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2_BRcatcount.csv", index = None, header=True)
df_IKEA.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2_IKcatcount.csv", index = None, header=True)
df_CARREFOUR.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH2\df\df_HH2_CAcatcount.csv", index = None, header=True)